In [1]:
import duckdb
con = duckdb.connect()


In [2]:
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("SET home_directory='..';")

con.execute("CREATE TABLE fact_prod AS SELECT * FROM read_csv_auto('../data/fact/fact_production.csv')")
con.execute("CREATE TABLE fact_down AS SELECT * FROM read_csv_auto('../data/fact/fact_downtime.csv')")
con.execute("CREATE TABLE dim_date AS SELECT * FROM read_csv_auto('../data/dim/dim_date.csv')")
con.execute("CREATE TABLE dim_machine AS SELECT * FROM read_csv_auto('../data/dim/dim_machine.csv')")
con.execute("CREATE TABLE dim_product AS SELECT * FROM read_csv_auto('../data/dim/dim_product.csv')")


In [3]:
con.execute("""
SELECT 
  m.machine_id,
  d.year,
  d.month,
  ROUND(AVG(
    ((p.planned_minutes - COALESCE(down.minutes_down, 0)) / p.planned_minutes) *
    (p.units_good / p.performance_units) *
    (p.units_good / p.units_total)
  ), 3) AS oee
FROM fact_prod p
LEFT JOIN fact_down down ON p.date_sk = down.date_sk AND p.machine_sk = down.machine_sk
JOIN dim_date d ON p.date_sk = d.date_sk
JOIN dim_machine m ON p.machine_sk = m.machine_sk
GROUP BY m.machine_id, d.year, d.month
ORDER BY m.machine_id, d.year, d.month
""").df()


,machine_id,year,month,oee
0,M1,2023,1,0.378
1,M1,2023,2,0.583
2,M1,2023,3,0.378
3,M2,2023,1,0.383
4,M2,2023,2,0.450
5,M2,2023,3,0.448
6,M3,2023,1,0.461
7,M3,2023,2,0.466
8,M3,2023,3,0.370


In [5]:
con.execute("SELECT COUNT(DISTINCT sku) FROM dim_product").df()


,count(DISTINCT sku)
0,3


In [6]:
con.execute("SELECT COUNT(DISTINCT product_sk) FROM fact_prod").df()


,count(DISTINCT product_sk)
0,3


In [7]:
#top 5 des produits les plus performants
con.execute("""
SELECT 
  pr.sku,
  ROUND(AVG(p.units_good / p.performance_units), 3) AS avg_performance
FROM fact_prod p
JOIN dim_product pr ON p.product_sk = pr.product_sk
GROUP BY pr.sku
ORDER BY avg_performance DESC
LIMIT 3
""").df()


,sku,avg_performance
0,P1,0.508
1,P3,0.488
2,P2,0.440
